In [1]:
import dotenv

dotenv.load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [32]:
from langchain.agents import tool
from langchain_community.document_loaders.generic import GenericLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language
from langchain_openai import OpenAIEmbeddings

@tool
def get_parsed_python_file(path: str) -> int:
    """Returns the content of the python code file"""
    parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
    loader = GenericLoader.from_filesystem(path, glob="**/*", suffixes=[".py"], parser=parser)
    documents = loader.load()
    # splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON)
    # texts = splitter.split_documents(documents)
    # embeddings = OpenAIEmbeddings()
    # embedded_texts = [text.page_content for text in texts]
    return documents[0].page_content

get_parsed_python_file.invoke("../docs/sample.py")

tools = [get_parsed_python_file]

In [34]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful code documenting assistant, but don't know the user's source code",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [35]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [36]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [38]:
list(agent_executor.stream({"input": "Write a docstring for the source code defined in the file ../docs/sample.py"}))



> Entering new AgentExecutor chain...

Invoking: `get_parsed_python_file` with `{'path': '../docs/sample.py'}`


def addNumbers(a, b):
    sum = a + b
    return sumThe `addNumbers` function takes two numbers as input and returns their sum.

Parameters:
- `a` (int): The first number.
- `b` (int): The second number.

Returns:
- `sum` (int): The sum of `a` and `b`.

> Finished chain.


[{'actions': [OpenAIToolAgentAction(tool='get_parsed_python_file', tool_input={'path': '../docs/sample.py'}, log="\nInvoking: `get_parsed_python_file` with `{'path': '../docs/sample.py'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_hWJO7KMPuwfuhBoML0Q0mLz7', 'function': {'arguments': '{\n  "path": "../docs/sample.py"\n}', 'name': 'get_parsed_python_file'}, 'type': 'function'}]})], tool_call_id='call_hWJO7KMPuwfuhBoML0Q0mLz7')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_hWJO7KMPuwfuhBoML0Q0mLz7', 'function': {'arguments': '{\n  "path": "../docs/sample.py"\n}', 'name': 'get_parsed_python_file'}, 'type': 'function'}]})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='get_parsed_python_file', tool_input={'path': '../docs/sample.py'}, log="\nInvoking: `get_parsed_python_file` with `{'path': '../docs/sample.py'}`\n\n\n", message_log=[AIMessageChunk(content='', 

In [41]:
list(agent_executor.stream({"input": "Write a docs for the file ../docs/sample.py"}))



> Entering new AgentExecutor chain...

Invoking: `get_parsed_python_file` with `{'path': '../docs/sample.py'}`


def addNumbers(a, b):
    sum = a + b
    return sum## File: sample.py

This file contains a function `addNumbers` that takes two numbers as input and returns their sum.

### Function: addNumbers(a, b)

This function takes two parameters `a` and `b`, which are numbers. It calculates the sum of `a` and `b` and returns the result.

#### Parameters

- `a` (number): The first number.
- `b` (number): The second number.

#### Returns

- `sum` (number): The sum of `a` and `b`.

> Finished chain.


[{'actions': [OpenAIToolAgentAction(tool='get_parsed_python_file', tool_input={'path': '../docs/sample.py'}, log="\nInvoking: `get_parsed_python_file` with `{'path': '../docs/sample.py'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_5PqgsAwpIF2mti39JapOKlBf', 'function': {'arguments': '{\n  "path": "../docs/sample.py"\n}', 'name': 'get_parsed_python_file'}, 'type': 'function'}]})], tool_call_id='call_5PqgsAwpIF2mti39JapOKlBf')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_5PqgsAwpIF2mti39JapOKlBf', 'function': {'arguments': '{\n  "path": "../docs/sample.py"\n}', 'name': 'get_parsed_python_file'}, 'type': 'function'}]})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='get_parsed_python_file', tool_input={'path': '../docs/sample.py'}, log="\nInvoking: `get_parsed_python_file` with `{'path': '../docs/sample.py'}`\n\n\n", message_log=[AIMessageChunk(content='', 